In [1]:
import pandas as pd

In [2]:
class Unitig:
    def __init__(self, head, sequence, anomaly):
        self.length = len(sequence)
        self.head = head
        self.sequence = sequence
        self.anomaly = anomaly

    def __repr__(self):
        return f"{self.head}; len: {self.length} and A: {self.anomaly}"
    
    @staticmethod
    def extract(path, unitigs):
        file = open(path, 'w')
        for unitig in unitigs:
            file.write(unitig.head + '\n')
            file.write(unitig.sequence + '\n')
        file.close()

In [3]:
info_path = "/home/Users/ns58/KOMB2/Data/PRJNA878603_SraRunTable.txt"

bucket = {}  # disease length : [SRA IDs]

with open(info_path, "r") as file:
    lines = file.readlines()
    iterable = iter(lines)
    next(iterable)  # skip first entry
    for line in iterable:
        data = line[:-1].split(',')  # 0: SRA ID, 24: gender, 29: disease_length
        if len(data) < 30:
            print("ERROR")
            continue
        if data[29] not in bucket:
            bucket[data[29]] = []
        bucket[data[29]].append(data[0])

In [4]:
%%script echo 'Extracting unitigs for BLAST. In order to run, remove this line'

top_unitigs_for_blast = []

samples = bucket['Control'] + bucket['Long'] + bucket['Short']
sample_paths = [f"/home/Users/ns58/KOMB2/PRJNA878603-Output/{sample}-k51-l150-c2" for sample in samples]
for sample, path in zip(samples, sample_paths):
    df = pd.read_csv(f"{path}/kcore.tsv", sep='\t')
    anomaly_df = pd.read_csv(f"{path}/CoreA_anomaly.txt", sep='\t', names=['#VID', 'Score'])
    combined = pd.merge(df, anomaly_df, on='#VID')
    
    name_to_score = combined.set_index('Name')['Score'].to_dict()
    
    unitigs = []
    
    file = open(f"{path}/combined.fasta", 'r')
    lines = file.readlines()
    file.close()
    
    for idx in range(0, len(lines), 2):
        first_part = lines[idx].split('|')[0]
        name = int(first_part.split('_')[1])
        
        sequence = lines[idx + 1][:-1]
        
        unitigs.append(Unitig(f">Unitig_{name}, from sample {sample}", sequence, name_to_score[name]))
        
    unitigs.sort(key= lambda unitig:unitig.anomaly, reverse=True)
    
    top_unitigs_for_blast.extend(unitigs[:10])

Unitig.extract("/home/Users/mt102/shared/KOMB-revision/BLAST-unitigs.fasta", top_unitigs_for_blast)


Extracting unitigs for BLAST. In order to run, remove this line


In [ ]:
reads_ct = []
unitigs_ct = []
filtered_ct = []
conditions = []
anomalous_percent = []

for sample, path in zip(samples, sample_paths):
    